In [7]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

SEED = 486
np.random.seed(486)

model = gp.Model("Scheduling")

# 인덱스 및 파라미터 정의
ALPHA_S = 0.05
ALPHA_E = 0.1
BETA = 0.05
Stages = 4
Machines = [3, 2, 3, 2]  # 각 스테이지별 기계 수
Products = 3
Jobs_per_product = 5
Processing_time = [45, 60, 120, 60]
total_processing_time = sum(Processing_time)
Due_dates = [total_processing_time + np.rint(np.random.uniform(ALPHA_S * total_processing_time, ALPHA_E * total_processing_time)) 
             for _ in range(Products * Jobs_per_product)]
Q_time = {
    1: [60 + np.rint(np.random.uniform(0, BETA * 60)), 120 + np.rint(np.random.uniform(0, BETA * 120))],  # Product 1
    2: [60+120 + np.rint(np.random.uniform(0, BETA * (60+120)))],      # Product 2
    3: [60 + np.rint(np.random.uniform(0, BETA * 60))]        # Product 3
}
Q_time_ranges = {
    1: [(1, 2), (2, 3)],  # Product 1
    2: [(1, 3)],          # Product 2
    3: [(3, 4)]           # Product 3
}
Total_jobs = Products * Jobs_per_product
M = 1e6  # 큰 상수

# 변수 정의
# x[k, i, j]: 작업 j가 스테이지 k에서 기계 i에서 수행되는지 여부 (이진 변수)
x = model.addVars(Stages, max(Machines), Total_jobs, vtype=GRB.BINARY, name="x")
# s[j, k]: 작업 j가 스테이지 k에서 시작하는 시간 (연속 변수)
s = model.addVars(Total_jobs, Stages, vtype=GRB.CONTINUOUS, name="s")
# c[j, k]: 작업 j가 스테이지 k에서 완료되는 시간 (연속 변수)
c = model.addVars(Total_jobs, Stages, vtype=GRB.CONTINUOUS, name="c")
# T[j]: 작업 j가 지각했는지 여부 (이진 변수)
T = model.addVars(Total_jobs, vtype=GRB.BINARY, name="T")
# Q[j]: 작업 j가 q-time 제한을 위반했는지 여부 (이진 변수)
Q = model.addVars(Total_jobs, vtype=GRB.BINARY, name="Q")
""" # same[j1, j2, i, k]: 작업 j1, j2가 스테이지 k에서 같은 기계 i에서 수행되는지 여부 (이진 변수)
same = model.addVars(Total_jobs,Total_jobs,Stages,max(Machines), vtype=GRB.BINARY, name="same") """

# 목적 함수
w = 0.5  # 가중치
model.setObjective(gp.quicksum(w *T[j] + (1 - w) * Q[j] for j in range(Total_jobs)), GRB.MINIMIZE)

# 제약 조건
for j in range(Total_jobs):
    for k in range(Stages):
        # 각 작업은 각 스테이지에서 하나의 기계에서만 수행
        model.addConstr(gp.quicksum(x[k, i, j] for i in range(Machines[k])) == 1)
        
        # 작업마다 각 스테이지의 완료 시간은 시작 시간 + 작업 시간
        model.addConstr(c[j, k] == s[j, k] + Processing_time[k])

# 다음 스테이지의 시작 시간은 이전 스테이지의 완료 시간 이후
for j in range(Total_jobs):
    for k in range(1, Stages):
        model.addConstr(s[j, k] >= c[j, k-1])

""" # 같은 스테이지 k에서 같은 기계 i에서 수행되면 same[j1, j2, k, i]가 1로 설정
for k in range(Stages):
    for i in range(Machines[k]):
        for j1 in range(Total_jobs):
            for j2 in range(j1 + 1, Total_jobs):
                model.addConstr(same[j1, j2, k, i] == gp.max_(x[k, i, j1] + x[k, i, j2] - 1, constant=0)) """



# 모든 시점에서 하나의 기계는 하나의 작업만 수행 가능
for k in range(Stages):
    for i in range(Machines[k]):
        for j1 in range(Total_jobs):
            for j2 in range(j1 + 1, Total_jobs):
                # 이진 변수 추가 - 작업 j1이 작업 j2보다 먼저 수행되거나, 그 반대인 경우
                z = model.addVar(vtype=GRB.BINARY, name=f"z_{k}_{i}_{j1}_{j2}")
                # same이 1인 작업 j1과 j2는 j1이 수행되는 동안 j2가 수행되지 않거나, j2가 수행되는 동안 j1이 수행되지 않아야 함
                model.addConstr(s[j1, k] >= c[j2, k] - M * (1 - x[k, i, j1]) - M * (1 - x[k, i, j2]) - M * z)
                model.addConstr(s[j2, k] >= c[j1, k] - M * (1 - x[k, i, j1]) - M * (1 - x[k, i, j2]) - M * (1 - z))

# Q-time 제약 조건
for j in range(Total_jobs):
    product_type = (j // Jobs_per_product) + 1
    if product_type in Q_time:
        for idx, (start, end) in enumerate(Q_time_ranges[product_type]):
            Q_time_limit = Q_time[product_type][idx]
            # Q-time 위반 시 Q[j]가 1로 설정
            model.addConstr((s[j, end-1] - c[j, start-1]) <= Q_time_limit + Q[j] * M)

# Tardy 제약 조건
for j in range(Total_jobs):
    due_date = Due_dates[j]
    # due_date 이후에 작업이 완료되면 T[j]가 1로 설정
    model.addConstr(c[j, Stages-1] <= due_date + T[j] * M)

# Stage 2, 4에는 3번째 기계가 없음
for j in range(Total_jobs):
    model.addConstr(x[1, 2, j] == 0)
    model.addConstr(x[3, 2, j] == 0)

model.optimize()

# 결과 출력
if model.Status == GRB.OPTIMAL:
    print('Optimal Objective Value: ', model.ObjVal)

for v in model.getVars():
    if v.x > 0:  # 0이 아닌 값만 출력
        print(f"{v.varName} {v.x}")

T_sum = sum(T[j].x for j in range(Total_jobs))
Q_sum = sum(Q[j].x for j in range(Total_jobs))
print(f"Number of Tardy Jobs: {T_sum}")
print(f"Number of Q-time Violations: {Q_sum}")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 2330 rows, 4080 columns and 10980 nonzeros
Model fingerprint: 0xb370271a
Variable types: 120 continuous, 3960 integer (3960 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Presolve removed 120 rows and 2820 columns
Presolve time: 0.00s
Presolved: 2210 rows, 1260 columns, 10770 nonzeros
Variable types: 60 continuous, 1200 integer (1200 binary)
Found heuristic solution: objective 12.0000000

Root relaxation: objective 0.000000e+00, 315 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# 최적화 결과에서 작업 스케줄을 추출하여 job_schedule 리스트에 저장
job_schedule = []
for j in range(Total_jobs):
    for k in range(Stages):
        for i in range(Machines[k]):
            if x[k, i, j].x > 0:
                start_time = s[j, k].x
                end_time = c[j, k].x
                job_schedule.append((j, k, i, start_time, end_time))

fig, ax = plt.subplots(figsize=(18, 8))

# 색상 매핑
cmap = plt.get_cmap("tab20")
colors = [cmap(i) for i in np.linspace(0, 1, Total_jobs)]

# 각 작업을 Gantt 차트에 추가
for (job, stage, machine, start, end) in job_schedule:
    color = colors[job % len(colors)]
    ax.add_patch(mpatches.Rectangle((start, machine + stage * (max(Machines) + 1)),
                                    end - start, 0.8, facecolor=color))
    ax.text(start + (end - start) / 2, machine + stage * (max(Machines) + 1) + 0.4, f'Product {(job // Jobs_per_product) + 1} - {job % Jobs_per_product + 1}', 
            ha='center', va='center', color='white', fontsize=6)

# y축과 x축 설정
y_ticks = [i + stage * (max(Machines) + 1) for stage in range(Stages) for i in range(Machines[stage])]
y_labels = [f'S{stage+1}-M{i+1}' for stage in range(Stages) for i in range(Machines[stage])]
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_labels)
ax.set_xlabel('Time')
ax.set_ylabel('Machine')
ax.set_title('Gantt Chart', fontsize=10)

max_time = max(end for (_, _, _, _, end) in job_schedule)
ax.set_xlim(0, max_time)
ax.set_ylim(0, max(y_ticks) + 1)

ax.grid(True)
plt.tight_layout()
plt.show()
